In [21]:
import import_ipynb
from advanced_rag import *
import json
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
import pandas as pd

## Creating necessery variables such as vectore base and declaring a llm

In [22]:
db, llm = initialize_rag()

KeyboardInterrupt: 

In [ ]:
try:
    print("llm.model_name:", llm.model_name)
except AttributeError:
    print("llm.model:", llm.model)

llm.model: gemini-2.0-flash


## Create Evaluation Dataset

We are building a dataset for RAGAS evaluation using questions and ground truth answers from `qa_eval.json`.

For each entry, we will include:
- **user_input**: the question
- **response**: the answer generated by the LLM using retrieved context
- **retrieved_contexts**: the documents returned by the retriever
- **reference**: the ground truth answer

In [16]:
def load_eval_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        eval_data = json.load(file)
        queries = [entry["question"] for entry in eval_data]
        ground_truth = [entry["ground_truth"] for entry in eval_data]
    return queries, ground_truth

def create_evaluation_dataset(answers, contexts, queries, ground_truth):
    eval_dataset = []
    for i in range(len(queries)):
        eval_dataset.append({
            "user_input": queries[i],
            "response": answers[i],
            "retrieved_contexts": contexts[i],
            "reference": ground_truth[i]
        })
    return eval_dataset

def extract_and_print_contexts(result):
    contexts = [doc.page_content for doc in result.get("source_documents", [])]
    return contexts

def run_queries(rag_chain, file_path):
    results = []
    contexts = []
    queries, ground_truth = load_eval_data(file_path)
    for query in queries:
        result = rag_chain({"question": query, "chat_history": []})
        results.append(result['answer'])
        contexts.append(extract_and_print_contexts(result))
    return create_evaluation_dataset(results, contexts, queries, ground_truth)

In [17]:
def evaluate_dataset(evaluation_dataset, llm):
    evaluator_llm = LangchainLLMWrapper(llm)
    print(type(evaluator_llm))
    print(evaluator_llm)
    result = evaluate(
        dataset=evaluation_dataset,
        metrics=[
            LLMContextRecall(), 
            Faithfulness(), 
            FactualCorrectness()
        ],
        llm=evaluator_llm,
    )
    return result

In [18]:

def display_eval_dataset_as_table(dataset, max_text_len=1000):
    def truncate(text):
        return text if len(text) <= max_text_len else text[:max_text_len] + "..."

    table_data = []
    for entry in dataset:
        table_data.append({
            "Question": truncate(entry["user_input"]),
            "Response": truncate(entry["response"]),
            "Context (1st)": truncate(entry["retrieved_contexts"][0] if entry["retrieved_contexts"] else ""),
            "Ground Truth": truncate(entry["reference"])
        })

    df = pd.DataFrame(table_data)
    return df


In [19]:
rag_chain, rag_memory = create_rag_chain(
    vectorstore=db,
    llm=llm,
    prompt_type='react',
    use_memory=True,
    retriever_k=3,
    use_reranking=False,
    top_k_chunks=10)

In [20]:
file_path = 'qa_eval.json'
dataset = run_queries(rag_chain, file_path)
eval_dataset = EvaluationDataset.from_list(dataset)
results = evaluate_dataset(eval_dataset, llm)
print(results)

<class 'ragas.llms.base.LangchainLLMWrapper'>
LangchainLLMWrapper(langchain_llm=GoogleGenerativeAI(...))


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
].
Retrying langchain_google_gena

KeyboardInterrupt: 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 49
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 49
}
].


In [ ]:
df = display_eval_dataset_as_table(dataset)
df

,🧠 Question,✅ Response,📚 Context (1st),🎯 Ground Truth
0,What is a Pokémon EX card?,Question: What is a Pokémon EX card?\nThought:...,28\nPokémon T rading Card Game Rules\nAPPENDIX...,Pokémon-EX cards are a special kind of card th...
1,What does haste do in Magic: The Gathering?,Question: What does haste do in Magic: The Gat...,702.8b Multiple instances of flash on the same...,Haste is a keyword that allows a creature to a...
2,How can a player build a hotel in Monopoly?,Question: How can a player build a hotel in Mo...,HOTELS...When a player has four houses on each...,A player must have four houses on every proper...
